In [ ]:
import pandas as pd
df_post_orgs = pd.read_csv("person.csv")
df_votes = pd.read_csv("voter.csv")
df_bills = pd.read_csv("bills.csv")

## **Import person data**

In [ ]:
import requests
import time
import pandas as pd

url = 'https://politigraph.wevis.info/graphql'
headers = {"Content-Type": "application/json"}

# query = """
# query People($limit: Int, $offset: Int) {
#   people(limit: $limit, offset: $offset) {
#     id
#     prefix
#     name
#     gender
#     national_identity
#     birth_date
#     educations
#     previous_occupations
#     memberships {
#       label
#       province
#       district_number
#       start_date
#       end_date
#       posts {
#         label
#         start_date
#         end_date
#         organizations{
#           name
#         }
#       }
#     }
#   }
# }
# """

query = """
query People($limit: Int, $offset: Int) {
  people(limit: $limit, offset: $offset) {
    id
    prefix
    name
    image
    gender
    national_identity
    birth_date
    educations
    previous_occupations
    memberships {
      label
      province
      district_number
      start_date
      end_date
      posts {
        label
        start_date
        end_date
        organizations{
          name
          image
          color
        }
      }
    }
  }
}
"""

def run_gql(query, variables):
  r = requests.post(url, headers=headers, json={"query":query, "variables":variables})
  r.raise_for_status()
  payload = r.json()
  if "error" in payload:
    raise RuntimeError(payload["error"])
  return payload["data"]

def fetch_all_people(batch_size=200, max_empty_loops=2, sleep_sec=1):
  seen_ids = set()
  all_rows = []
  offset = 0
  stalled = 0

  while True:
    data = run_gql(query, {"limit": batch_size, "offset": offset})
    page = data["people"] or []
    page_size = len(page)

    print(f"offset={offset} get={page_size}")
    if page_size == 0:
      print("No more data.")
      break

    new_rows = [row for row in page if row.get("id") not in seen_ids]
    for row in new_rows:
      seen_ids.add(row["id"])
    all_rows.extend(new_rows)

    print(f"added new ={len(new_rows)} total={len(all_rows)}")

    if len(new_rows) == 0:
      stalled += 1
      if stalled >= max_empty_loops:
        print("Stalled pagination. Stopping.")
    else:
      stalled = 0

    if page_size < batch_size:
      print("Likely last page.")
      break

    offset += batch_size
    time.sleep(sleep_sec)
  return all_rows


people = fetch_all_people(batch_size=100)
df = pd.json_normalize(people, sep="__")
print(df.shape)



HTTPError: 502 Server Error: Bad Gateway for url: https://politigraph.wevis.info/graphql

In [ ]:
df_posts = pd.json_normalize(
    data=people,
    record_path=["memberships", "posts"],
    meta=[
        ["memberships", "label"],
        ["memberships", "province"],
        ["memberships", "district_number"],
        ["memberships", "start_date"],
        ["memberships", "end_date"],
        "prefix", "name", "gender", "image", "national_identity", "birth_date", "educations", "previous_occupations"
    ],
    sep="__",
    record_prefix="post__",
    meta_prefix="m_or_person__"
).rename(columns={
    "m_or_person__memberships__label": "m__label",
    "m_or_person__memberships__province": "m__province",
    "m_or_person__memberships__district_number": "m__district_number",
    "m_or_person__memberships__start_date": "m__start_date",
    "m_or_person__memberships__end_date": "m__end_date",
    "m_or_person__prefix": "person__prefix",
    "m_or_person__name": "person__name",
    "m_or_person__gender": "person__gender",
    "m_or_person__national_identity": "person__national_identity",
    "m_or_person__birth_date": "person__birth_date",
    "m_or_person__educations": "person__educations",
    "m_or_person__previous_occupations": "person__previous_occupations",
    "m_or_person__image": "person__image"
})


df_post_orgs = df_posts.explode("post__organizations", ignore_index=True)
org_cols = pd.json_normalize(df_post_orgs["post__organizations"]).add_prefix("org__")
df_post_orgs = pd.concat(
    [df_post_orgs.drop(columns=["post__organizations"]), org_cols],
    axis=1
)

new_col_order = ['person__prefix','person__name', 'person__image', 'person__gender','person__national_identity'
,'person__birth_date','person__educations','person__previous_occupations','post__label'
,'post__start_date','post__end_date','m__label','m__province','m__district_number'
,'m__start_date','m__end_date','org__name', 'org__image', 'org__color']
df_post_orgs = df_post_orgs[new_col_order]
df_post_orgs.to_csv("person.csv")

## **Import voting data**

In [ ]:
import requests
import time
import pandas as pd

url = 'https://politigraph.wevis.info/graphql'
headers = {"Content-Type": "application/json"}

query = """
query VoteEvents($limit: Int, $offset: Int) {
  voteEvents(limit: $limit, offset: $offset) {
    id
    title
    nickname
    classification
    publish_status
    start_date
    end_date
    pass_condition
    result
    agree_count
    disagree_count
    abstain_count
    novote_count
    votes {
      option
      voters {
        name
      }
    }
  }
}
"""

def run_gql(query, variables):
  r = requests.post(url, headers=headers, json={"query":query, "variables":variables})
  r.raise_for_status()
  payload = r.json()
  if "error" in payload:
    raise RuntimeError(payload["error"])
  return payload["data"]

def fetch_all(batch_size=200, max_empty_loops=2, sleep_sec=1):
  seen_ids = set()
  all_rows = []
  offset = 0
  stalled = 0

  while True:
    data = run_gql(query, {"limit": batch_size, "offset": offset})
    page = data["voteEvents"] or []
    page_size = len(page)

    print(f"offset={offset} get={page_size}")
    if page_size == 0:
      print("No more data.")
      break

    new_rows = [row for row in page if row.get("id") not in seen_ids]
    for row in new_rows:
      seen_ids.add(row["id"])
    all_rows.extend(new_rows)

    print(f"added new ={len(new_rows)} total={len(all_rows)}")

    if len(new_rows) == 0:
      stalled += 1
      if stalled >= max_empty_loops:
        print("Stalled pagination. Stopping.")
    else:
      stalled = 0

    if page_size < batch_size:
      print("Likely last page.")
      break

    offset += batch_size
    time.sleep(sleep_sec)
  return all_rows


voting = fetch_all(batch_size=100)
df = pd.json_normalize(voting, sep="__")
print(df.shape)



In [ ]:
df_votes = pd.json_normalize(
    data=voting,
    record_path=["votes", "voters"],
    meta=[
        ["votes", "option"],
        "title", "nickname", "classification", "publish_status", "start_date", "end_date", "pass_condition"
        , "result", "agree_count", "disagree_count", "abstain_count", "novote_count"
    ],
    sep="__",
    record_prefix="voters__",
    meta_prefix="evt__"
).rename(columns={
    "evt__votes__option": "vote__option",
    "evt__title": "event__title",
    "evt__nickname": "event__nickname",
    "evt__classification": "event__classification",
    "evt__publish_status": "event__publish_status",
    "evt__start_date": "event__start_date",
    "evt__end_date": "event__end_date",
    "evt__pass_condition": "event__pass_condition",
    "evt__result": "event__result",
    "evt__agree_count": "event__agree_count",
    "evt__disagree_count": "event__disagree_count",
    "evt__abstain_count": "event__abstain_count",
    "evt__novote_count": "event__novote_count",
})

new_col_order = ['event__title', 'event__nickname',
       'event__classification', 'event__publish_status', 'event__start_date',
       'event__end_date', 'event__pass_condition', 'event__result',
       'event__agree_count', 'event__disagree_count', 'event__abstain_count',
       'event__novote_count', 'voters__name', 'vote__option',]
df_votes = df_votes[new_col_order]
df_votes.to_csv("voter.csv")

## **Import bills data**

In [ ]:
import requests
import time
import pandas as pd

url = 'https://politigraph.wevis.info/graphql'
headers = {"Content-Type": "application/json"}

query = """
query Bills($limit: Int, $offset: Int) {
  bills(limit: $limit, offset: $offset) {
    id
    title
    nickname
    status
    categories
    classification
    proposal_date
    creators {
      ... on Person {
        name
      }
    }
  }
}
"""

def run_gql(query, variables):
  r = requests.post(url, headers=headers, json={"query":query, "variables":variables})
  r.raise_for_status()
  payload = r.json()
  if "error" in payload:
    raise RuntimeError(payload["error"])
  return payload["data"]

def fetch_all(batch_size=200, max_empty_loops=2, sleep_sec=1):
  seen_ids = set()
  all_rows = []
  offset = 0
  stalled = 0

  while True:
    data = run_gql(query, {"limit": batch_size, "offset": offset})
    page = data["bills"] or []
    page_size = len(page)

    print(f"offset={offset} get={page_size}")
    if page_size == 0:
      print("No more data.")
      break

    new_rows = [row for row in page if row.get("id") not in seen_ids]
    for row in new_rows:
      seen_ids.add(row["id"])
    all_rows.extend(new_rows)

    print(f"added new ={len(new_rows)} total={len(all_rows)}")

    if len(new_rows) == 0:
      stalled += 1
      if stalled >= max_empty_loops:
        print("Stalled pagination. Stopping.")
    else:
      stalled = 0

    if page_size < batch_size:
      print("Likely last page.")
      break

    offset += batch_size
    time.sleep(sleep_sec)
  return all_rows


bills = fetch_all(batch_size=100)
df = pd.json_normalize(bills, sep="__")
print(df.shape)

In [ ]:
df_bills = pd.json_normalize(
    data=bills,
    record_path=["creators"],
    meta=[
        "id", "title", "nickname", "status", "categories", "classification", "proposal_date",
    ],
    sep="__",
    record_prefix="creators__",
    meta_prefix="bill__"
)

new_col_order = ['bill__id', 'bill__title', 'bill__nickname',
       'bill__status', 'bill__categories', 'bill__classification',
       'bill__proposal_date', 'creators__name']
df_bills = df_bills[new_col_order]
df_bills.to_csv("bills.csv")

## **Transform Data**

### **Dimension Table Person**

In [ ]:
#สร้าง dim รายชื่อ สส.
fil_rep = df_post_orgs[(df_post_orgs["post__label"].str.contains("สส. ชุดที่", case=False, na=False)) & (df_post_orgs["post__label"].str.len() == 13)]["post__label"].unique().max()
df_rep = df_post_orgs[(df_post_orgs["post__label"] == fil_rep) & (df_post_orgs["m__label"] == "แบ่งเขต") & df_post_orgs["m__end_date"].isna()]

#สร้าง dim พรรคแต่ละสส.
df_prep_party = df_post_orgs[(df_post_orgs["post__label"].str.contains("สมาชิกพรรค", case=False, na=False)) & (df_post_orgs["m__end_date"].isna())][["person__name", "post__label", "org__name", "org__image", "org__color", "m__start_date"]].drop_duplicates()
df_party = df_prep_party[df_prep_party["m__start_date"] == df_prep_party.groupby("person__name")["m__start_date"].transform("max")][["person__name", "post__label", "org__name",  "org__image", "org__color"]].rename(columns={"post__label": "member_of", "org__name": "party_name", "org__image": "party_image", "org__color": "party_color"})

#สร้าง final table ของสส.
df_person = pd.merge(df_rep, df_party, on="person__name", how="left")
df_person = df_person[["person__prefix", "person__name", "person__image", "member_of", "party_image", "party_color"]].rename(columns={"person__prefix": "prefix", "person__name": "person_name", "person__image": "image"})

#ตัวอย่าง df_person
df_person.head()


,prefix,person_name,image,member_of,party_image,party_color
0,นาย,สยาม หัตถสงเคราะห์,https://politigraph.wevis.info/assets/people/ส...,สมาชิกพรรคเพื่อไทย,https://politigraph.wevis.info/assets/organiza...,#F41724
1,นาย,สยาม เพ็งทอง,NaN,สมาชิกพรรคภูมิใจไทย,https://politigraph.wevis.info/assets/organiza...,#2C3487
2,นาย,สรชัด สุจิตต์,https://politigraph.wevis.info/assets/people/ส...,สมาชิกพรรคชาติไทยพัฒนา,https://politigraph.wevis.info/assets/organiza...,#FF5BD1
3,นาย,สรพัช ศรีปราชญ์,NaN,สมาชิกพรรคประชาชน,https://politigraph.wevis.info/assets/organiza...,#FF5C00
4,นาย,สรรพภัญญู ศิริไปล์,NaN,สมาชิกพรรคเพื่อไทย,https://politigraph.wevis.info/assets/organiza...,#F41724


### **Dimension Table Person Votes**

In [ ]:
#สร้าง dimenstion table การโหวตแต่ละประเภทของสส.
df_person_vote = df_votes[df_votes["event__start_date"].str[:4] == "2025"].groupby(["voters__name", "vote__option"]).agg(no_of_option = ("vote__option", "count")).reset_index()
df_person_vote = df_person_vote.rename(columns={"voters__name": "person_name", "vote__option": "option"})
df_person_vote = df_person_vote[df_person_vote["option"].isin(["เห็นด้วย", "ไม่เห็นด้วย", "งดออกเสียง", "ลา / ขาดลงมติ", "ไม่ลงคะแนน"])]

#ตัวอย่าง
df_person_vote.head()




,person_name,option,no_of_option
0,กนก ลิ้มตระกูล,ลา / ขาดลงมติ,1
1,กนก วงษ์ตระหง่าน,ลา / ขาดลงมติ,1
2,กนกวรรณ วิลาวัลย์,ลา / ขาดลงมติ,1
3,กนิษฐ์ ชาญปรีชญา,ลา / ขาดลงมติ,1
4,กมนทรรศน์ กิตติสุนทรสกุล,งดออกเสียง,3


### **Dimension Table Province**

In [ ]:
#สร้าง dim จังหวัดว่ามีสส.กี่คน
df_province = df_person.groupby("m__province").agg(number_of_member = ("person__name", "count"))

#ตัวอย่าง df_province
df_province.head()

,number_of_member
m__province,
กระบี่,3
กรุงเทพมหานคร,33
กาญจนบุรี,5
กาฬสินธ์ุ,6
กำแพงเพชร,4


### **Fact Vote event**

In [ ]:
df_fact_votes = df_votes[df_votes["event__start_date"].str[:4] == "2025"][["event__title", "event__start_date", "event__end_date", "voters__name", "vote__option"]]
df_fact_votes = pd.merge(df_fact_votes, df_person, left_on="voters__name", right_on="person__name", how="left")




In [ ]:
#สร้างสรุปผลโหวตมติแต่ละจังหวัด
df_fact_votes = df_fact_votes[~(df_fact_votes["person__name"].isna()) & (df_fact_votes["event__title"] != "การพิจารณาให้ความเห็นชอบบุคคลซึ่งสมควรได้รับแต่งตั้งเป็นนายกรัฐมนตรี ตามมาตรา ๑๕๙ ของรัฐธรรมนูญแห่งราชอาณาจักรไทย")].groupby(["event__title", "m__province", "vote__option"]).size().reset_index(name="no_of_option")

df_fact_votes = df_fact_votes[df_fact_votes["no_of_option"] == df_fact_votes.groupby(["event__title", "event__start_date", "m__province"])["no_of_option"].transform("max")]



# ถ้าเสียงเท่ากันให้สรุปว่า ผลโหวดตเสมอ (เฉพาะเห็นด้วยและไม่เห็นด้วย)
g = df_fact_votes.groupby(["event__title", "event__start_date", "m__province"])["vote__option"]
size = g.transform("size")
has_yes = g.transform(lambda s: (s == "เห็นด้วย").any())
has_no = g.transform(lambda s: (s == "ไม่เห็นด้วย").any())
tie_mask = (size > 1) & has_yes & has_no

df_fact_votes["result"] = df_fact_votes["vote__option"]
df_fact_votes.loc[tie_mask, "result"] = "ผลโหวตเสมอ"

df_fact_votes = df_fact_votes[["event__title", "event__start_date", "m__province", "result"]].drop_duplicates()


# ถ้ามี ลา / ขาดลงมติ ลบออก
g = df_fact_votes.groupby(["event__title", "event__start_date", "m__province"])["result"]
size = g.transform("size")

tie_mask = (size > 1) & (df_fact_votes["result"] == "ลา / ขาดลงมติ")

df_fact_votes.loc[tie_mask, "result"] = "ไม่เอา"
df_fact_votes = df_fact_votes[df_fact_votes["result"] != "ไม่เอา"]
df_fact_votes = df_fact_votes[["event__title", "event__start_date", "m__province", "result"]].drop_duplicates()


# ถ้ามี งดออกเสียง ลบออก
g = df_fact_votes.groupby(["event__title", "event__start_date", "m__province"])["result"]
size = g.transform("size")
tie_mask = (size > 1) & (df_fact_votes["result"].isin(["งดออกเสียง"]))

df_fact_votes.loc[tie_mask, "result"] = "ไม่เอา"
df_fact_votes = df_fact_votes[df_fact_votes["result"] != "ไม่เอา"]
df_fact_votes = df_fact_votes[["event__title", "event__start_date", "m__province", "result"]].drop_duplicates()


# ถ้ามี ไม่ลงคะแนนเสียง ลบออก
g = df_fact_votes.groupby(["event__title", "event__start_date", "m__province"])["result"]
size = g.transform("size")
tie_mask = (size > 1) & (df_fact_votes["result"].isin(["ไม่ลงคะแนนเสียง"]))

df_fact_votes.loc[tie_mask, "result"] = "ไม่เอา"
df_fact_votes = df_fact_votes[df_fact_votes["result"] != "ไม่เอา"]
df_fact_votes = df_fact_votes[["event__title", "event__start_date", "m__province", "result"]].drop_duplicates()

#บางมติมีจังหวัดโหวตไม่ครบ ไม่รู้เพราะอะไร

# ตัวอย่าง
df_fact_votes.head()


,event__title,m__province,result
0,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กระบี่,เห็นด้วย
2,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กรุงเทพมหานคร,เห็นด้วย
4,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กาญจนบุรี,เห็นด้วย
5,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กาฬสินธ์ุ,เห็นด้วย
6,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กำแพงเพชร,ลา / ขาดลงมติ


In [ ]:
df_fact_portion = df_votes[df_votes["event__start_date"].str[:4] == "2025"][["event__title", "event__start_date", "event__end_date", "voters__name", "vote__option"]]
df_fact_portion = pd.merge(df_fact_portion, df_person, left_on="voters__name", right_on="person__name", how="left")
df_fact_portion = df_fact_portion[~(df_fact_portion["person__name"].isna()) & (df_fact_portion["event__title"] != "การพิจารณาให้ความเห็นชอบบุคคลซึ่งสมควรได้รับแต่งตั้งเป็นนายกรัฐมนตรี ตามมาตรา ๑๕๙ ของรัฐธรรมนูญแห่งราชอาณาจักรไทย")].groupby(["event__title", "m__province", "vote__option"]).size().reset_index(name="no_of_option")


In [ ]:
df_fact_portion["use_a_right"] = df_fact_portion["vote__option"].apply(lambda x: "ไม่ใช้สิทธิ์" if x == "ลา / ขาดลงมติ" else "ใช้สิทธิ์")
df_fact_portion = df_fact_portion.groupby(["event__title", "m__province", "use_a_right"]).agg(no_of_person = ("no_of_option", "sum")).reset_index()

In [ ]:
df_fact_total = df_fact_portion.groupby(["event__title", "m__province"])["no_of_person"].transform("sum")

df_fact_portion["portion_use_right"] = df_fact_portion["no_of_person"] / df_fact_total

g = df_fact_portion.groupby(["event__title", "m__province"])["use_a_right"]
size = g.transform("size")
tie_mask = (size > 1) & (df_fact_portion["use_a_right"] == "ไม่ใช้สิทธิ์")

df_fact_portion.loc[tie_mask, "use_a_right"] = "ไม่เอา"
df_fact_portion = df_fact_portion[df_fact_portion["use_a_right"] != "ไม่เอา"]

In [ ]:
df_fact_votes = pd.merge(df_fact_votes, df_fact_portion, on=["event__title", "m__province"], how="left")
df_fact_votes.loc[(df_fact_votes["use_a_right"] == "ไม่ใช้สิทธิ์"), "portion_use_right"] = 0.0
df_fact_votes["type"] = "All"

In [ ]:
# ตัวอย่าง
df_fact_votes.head()

,event__title,m__province,result,use_a_right,no_of_person,portion_use_right,type
0,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กระบี่,เห็นด้วย,ใช้สิทธิ์,3,1.000000,All
1,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กรุงเทพมหานคร,เห็นด้วย,ใช้สิทธิ์,26,0.787879,All
2,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กาญจนบุรี,เห็นด้วย,ใช้สิทธิ์,4,0.800000,All
3,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กาฬสินธ์ุ,เห็นด้วย,ใช้สิทธิ์,6,1.000000,All
4,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กำแพงเพชร,ลา / ขาดลงมติ,ใช้สิทธิ์,1,0.250000,All


In [ ]:
df_fact_option = df_votes[df_votes["event__start_date"].str[:4] == "2025"][["event__title", "event__start_date", "event__end_date", "voters__name", "vote__option"]]
df_fact_option = pd.merge(df_fact_option, df_person, left_on="voters__name", right_on="person__name", how="left")
df_fact_option = df_fact_option[~(df_fact_option["person__name"].isna()) & (df_fact_option["event__title"] != "การพิจารณาให้ความเห็นชอบบุคคลซึ่งสมควรได้รับแต่งตั้งเป็นนายกรัฐมนตรี ตามมาตรา ๑๕๙ ของรัฐธรรมนูญแห่งราชอาณาจักรไทย")].groupby(["event__title", "m__province", "vote__option"]).size().reset_index(name="no_of_option")


In [ ]:
df_fact_size = df_fact_option.groupby(["event__title", "m__province"])["no_of_option"].transform("sum")

df_fact_option["portion"] = df_fact_option["no_of_option"] / df_fact_size
df_fact_option["type"] = df_fact_option["vote__option"]


In [ ]:
df_fact_votes = df_fact_votes[["event__title", "m__province", "result", "portion_use_right", "type"]].rename(columns={"event__title": "title", "m__province": "province", "result": "option", "portion_use_right": "portion"})
df_fact_option = df_fact_option[["event__title", "m__province", "vote__option", "portion", "type"]].rename(columns={"event__title": "title", "m__province": "province", "vote__option": "option"})

In [ ]:
df_fact = pd.concat([df_fact_votes, df_fact_option], ignore_index=True)

In [ ]:
df_fact.head()

,title,province,option,portion,type
0,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กระบี่,เห็นด้วย,1.000000,All
1,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กรุงเทพมหานคร,เห็นด้วย,0.787879,All
2,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กาญจนบุรี,เห็นด้วย,0.800000,All
3,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กาฬสินธ์ุ,เห็นด้วย,1.000000,All
4,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กำแพงเพชร,ลา / ขาดลงมติ,0.250000,All


### **Vote Detail**

In [ ]:
df_vote_detail = df_votes[df_votes["event__start_date"].str[:4] == "2025"][["event__title", "event__start_date", "event__end_date", "voters__name", "vote__option"]]
df_vote_detail = pd.merge(df_vote_detail, df_person, left_on="voters__name", right_on="person__name", how="left")
df_vote_detail = df_vote_detail[~(df_vote_detail["person__name"].isna()) & (df_vote_detail["event__title"] != "การพิจารณาให้ความเห็นชอบบุคคลซึ่งสมควรได้รับแต่งตั้งเป็นนายกรัฐมนตรี ตามมาตรา ๑๕๙ ของรัฐธรรมนูญแห่งราชอาณาจักรไทย")]
df_vote_detail = df_vote_detail[["event__title", "m__province", "voters__name", "vote__option"]].rename(columns={"event__title":"title", "m__province":"province", "voters__name": "person_name", "vote__option": "option"})

In [ ]:
df_vote_detail.head()

,title,province,person_name,option
0,ร่างข้อบังคับการประชุมรัฐสภา (ฉบับที่ ..) พ.ศ...,ระยอง,กมนทรรศน์ กิตติสุนทรสกุล,เห็นด้วย
203,ร่างข้อบังคับการประชุมรัฐสภา (ฉบับที่ ..) พ.ศ...,กรุงเทพมหานคร,ไชยามพวาน มั่นเพียรจิตต์,เห็นด้วย
204,ร่างข้อบังคับการประชุมรัฐสภา (ฉบับที่ ..) พ.ศ...,อุบลราชธานี,สมศักดิ์ บุญประชม,เห็นด้วย
205,ร่างข้อบังคับการประชุมรัฐสภา (ฉบับที่ ..) พ.ศ...,อุบลราชธานี,พิมพกาญจน์ พลสมัคร,เห็นด้วย
206,ร่างข้อบังคับการประชุมรัฐสภา (ฉบับที่ ..) พ.ศ...,ปราจีนบุรี,วุฒิพงศ์ ทองเหลา,ลา / ขาดลงมติ


## **Result**

In [ ]:
df_fact.to_json("fact_data.json", orient="records", force_ascii=False, indent=2)
df_fact.head(10)

,title,province,option,portion,type
0,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กระบี่,เห็นด้วย,1.000000,All
1,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กรุงเทพมหานคร,เห็นด้วย,0.787879,All
2,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กาญจนบุรี,เห็นด้วย,0.800000,All
3,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กาฬสินธ์ุ,เห็นด้วย,1.000000,All
4,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,กำแพงเพชร,ลา / ขาดลงมติ,0.250000,All
5,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,ขอนแก่น,เห็นด้วย,0.909091,All
6,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,จันทบุรี,เห็นด้วย,1.000000,All
7,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,ฉะเชิงเทรา,เห็นด้วย,0.750000,All
8,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,ชลบุรี,เห็นด้วย,0.900000,All
9,การพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญว่า...,ชัยนาท,เห็นด้วย,0.500000,All


In [ ]:
df_vote_detail.to_json("vote_detail_data.json", orient="records", force_ascii=False, indent=2)
df_vote_detail.head()

,title,province,person_name,option
0,ร่างข้อบังคับการประชุมรัฐสภา (ฉบับที่ ..) พ.ศ...,ระยอง,กมนทรรศน์ กิตติสุนทรสกุล,เห็นด้วย
203,ร่างข้อบังคับการประชุมรัฐสภา (ฉบับที่ ..) พ.ศ...,กรุงเทพมหานคร,ไชยามพวาน มั่นเพียรจิตต์,เห็นด้วย
204,ร่างข้อบังคับการประชุมรัฐสภา (ฉบับที่ ..) พ.ศ...,อุบลราชธานี,สมศักดิ์ บุญประชม,เห็นด้วย
205,ร่างข้อบังคับการประชุมรัฐสภา (ฉบับที่ ..) พ.ศ...,อุบลราชธานี,พิมพกาญจน์ พลสมัคร,เห็นด้วย
206,ร่างข้อบังคับการประชุมรัฐสภา (ฉบับที่ ..) พ.ศ...,ปราจีนบุรี,วุฒิพงศ์ ทองเหลา,ลา / ขาดลงมติ


In [ ]:
df_person.to_json("person_data.json", orient="records", force_ascii=False, indent=2)
df_person.head()

,prefix,person_name,image,member_of,party_image,party_color
0,นาย,สยาม หัตถสงเคราะห์,https://politigraph.wevis.info/assets/people/ส...,สมาชิกพรรคเพื่อไทย,https://politigraph.wevis.info/assets/organiza...,#F41724
1,นาย,สยาม เพ็งทอง,NaN,สมาชิกพรรคภูมิใจไทย,https://politigraph.wevis.info/assets/organiza...,#2C3487
2,นาย,สรชัด สุจิตต์,https://politigraph.wevis.info/assets/people/ส...,สมาชิกพรรคชาติไทยพัฒนา,https://politigraph.wevis.info/assets/organiza...,#FF5BD1
3,นาย,สรพัช ศรีปราชญ์,NaN,สมาชิกพรรคประชาชน,https://politigraph.wevis.info/assets/organiza...,#FF5C00
4,นาย,สรรพภัญญู ศิริไปล์,NaN,สมาชิกพรรคเพื่อไทย,https://politigraph.wevis.info/assets/organiza...,#F41724


In [ ]:
df_person_vote.to_json("person_vote_data.json", orient="records", force_ascii=False, indent=2)
df_person_vote.head()

,person_name,option,no_of_option
0,กนก ลิ้มตระกูล,ลา / ขาดลงมติ,1
1,กนก วงษ์ตระหง่าน,ลา / ขาดลงมติ,1
2,กนกวรรณ วิลาวัลย์,ลา / ขาดลงมติ,1
3,กนิษฐ์ ชาญปรีชญา,ลา / ขาดลงมติ,1
4,กมนทรรศน์ กิตติสุนทรสกุล,งดออกเสียง,3
